In [1]:
pip install pandas numpy openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np


file_path = "retaildata.xlsx"
df = pd.read_excel(file_path)

print(df.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


In [4]:
#Data Processing

print(df.isnull().sum()) #counts missing values per column

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [5]:
df.dropna(subset=['Description'])
df.dropna(subset=['CustomerID'])

df.drop_duplicates() #drop duplicate rows


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [ ]:
# TOTAL PRICE
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [8]:
#INVOICE DATE TO DATE TIME FORMAT

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


In [12]:
#RFM Metrics

latest_date = df['InvoiceDate'].max()

rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda date: (latest_date - date.max()).days, #Recency
    'InvoiceNo': 'count', # Frequency
    'TotalPrice': 'sum' #Monetary
})

rfm.columns = ['Recency','Frequency','Monetary']

print(rfm.head())

            Recency  Frequency  Monetary
CustomerID                              
12346.0         325          2      0.00
12347.0           1        182   4310.00
12348.0          74         31   1797.24
12349.0          18         73   1757.55
12350.0         309         17    334.40


In [14]:
#Normalize RFM Scores

rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

In [17]:
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# Define customer segments
def segment_customer(row):
    if row['RFM_Score'] >= '555':
        return 'Best Customer'
    elif row['RFM_Score'] >= '444':
        return 'Loyal Customer'
    elif row['RFM_Score'] >= '333':
        return 'Potential Loyalist'
    elif row['RFM_Score'] >= '222':
        return 'Need Attention'
    else:
        return 'Lost Customer'

rfm['Customer_Segment'] = rfm.apply(segment_customer, axis=1)

print(rfm[['RFM_Score', 'Customer_Segment']].head(10))  # Display first 10 rows

           RFM_Score    Customer_Segment
CustomerID                              
12346.0          111       Lost Customer
12347.0          555       Best Customer
12348.0          234      Need Attention
12349.0          444      Loyal Customer
12350.0          122       Lost Customer
12352.0          344  Potential Loyalist
12353.0          111       Lost Customer
12354.0          134       Lost Customer
12355.0          112       Lost Customer
12356.0          445      Loyal Customer


In [16]:
country_segments = df.groupby('Country')['TotalPrice'].sum().reset_index()
country_segments = country_segments.sort_values(by='TotalPrice', ascending=False)

print(country_segments.head())  # Show top spending countries


           Country   TotalPrice
36  United Kingdom  8187806.364
24     Netherlands   284661.540
10            EIRE   263276.820
14         Germany   221698.210
13          France   197403.900


In [27]:
# Reset index correctly
rfm = rfm.reset_index(drop=True)
country_segments = country_segments.reset_index(drop=True)

# Save to Excel
with pd.ExcelWriter("retaildata.xlsx") as writer:
    rfm.to_excel(writer, sheet_name="RFM Segmentation", index=False)  
    country_segments.to_excel(writer, sheet_name="Country Analysis", index=False)  


In [29]:
# Drop the first two columns
rfm = rfm.drop(columns=['level_0', 'index'], errors='ignore')

# Save to Excel
with pd.ExcelWriter("retaildata.xlsx") as writer:
    rfm.to_excel(writer, sheet_name="RFM Segmentation", index=False)  
    country_segments.to_excel(writer, sheet_name="Country Analysis", index=False)  
